In [133]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd

In [2]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [3]:
formats = pd.read_excel('invoice_formats.xlsx')
final = pd.read_excel('Extracted_Data.xlsx')

In [134]:
#Image = "Makro Invoices/Makro Invoices/Converted/02.08.21_Graciosamar Cruceros, S.L._97,5_page1.png"

In [135]:
%%time
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":'',
      "Invoice_Number":''}
img  = cv2.imread("Makro Invoices/Makro Invoices/Converted/02.08.21_Globalia Corporate Travel SLU_574,88_page8.png", cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
vat = []
date = []
print(type(text_list))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):


        if (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            print(b)
        elif (re.search("[0-9]{8}-[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}-[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
        elif (re.search("[0-9]{8}[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}[A-Z]{1}",a)
            dat["Vat Number"] = b
            vat.append(b)
    
        elif (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)

        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)
            dat["Vat Number"] = b[0]

        if len(vat)>1:
            dat["Vat Number"] = vat[0]
        else:
            pass
         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b[0]
            date.append(b)
        if (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b[0]
            date.append(b)
        if (re.search("\d{2}/\d{2}/\d{2}",a)):
            b = re.findall("\d{2}/\d{2}/\d{2}",a)
            dat["Date"] = b[0]
            date.append(b)
        if (re.search("[0-9]{2}.[0-9]{2}.[0-9]{4}",a)):
            b = re.findall("[0-9]{2}.[0-9]{2}.[0-9]{4}",a)
            dat["Date"] = b
            date.append(b)
        if len(date)>1:
            dat["Date"] = date[0]

        #invoice number section
        if ("Factura" in a or "ALBARAN" in a or "Factura: " in a):

            print(a)
            b = a.split(" ")
            if b[0] == "ALBARAN":
                dat["Invoice_Number"] = b[1]
            print(b)
            if len(b)>3:
                print(b)
                if len(b)>4 and b[2] !='—' and len(b[1])!=25 and len(b[1])>5:
                    dat["Invoice_Number"] = b[1]+b[2]
                    print(b)
                elif len(b)>=5 and len(b[1])==25:
                    dat["Invoice_Number"] = b[1]
                elif len(b)>6 and len(b[3])>15:
                    dat["Invoice_Number"] = b[1]+ b[2]+ b[3]
                elif len(b)>6 and b[1]== "0/0":
                    dat["Invoice_Number"] = b[1] + b[2]
                    
        if ("Tarjeta" in a or "Credito" in a or  "pagina" in a or "Total:" in a  or "TOTAL" in a or  "Total Factura" in a or "TOTAL FACTURA" in a or "Total CON IGIC" in a  or "IMPORTE FACTURA EU" in a or "IMPORTE FACTURA" in a or "EUROS" in a):
            print("Case Total")
            b = a.split(" ")
            print(b)
            if len(b[-1])==2:
                
                dat["Total"] = b[-2]+b[-1]
                print(b[-2]+b[-1])
                
            if b[-1]=='€':
                dat["Total"] = b[-2]
            if b[-1] == "|":
                dat["Total"] = b[-2]
            if b[-1] == "EUR":
                dat["Total"] = b[-2]
            else:
                dat["Total"] = b[-1]
                print(b[-1])
                
if dat["Total"] == 0 or len(dat['Total'])==1 or dat["Total"]=="SUB TOTAL"  :
        tot = []
        im = Image.open(r"Makro Invoices/Makro Invoices/Converted/02.08.21_Globalia Corporate Travel SLU_574,88_page8.png") 
        # Size of the image in pixels (size of orginal image) 
        # (This is not mandatory) 
        width, height = im.size 
        print(im.size)
        # Setting the points for cropped image 
        left = 605
        top = 600
        right = 1675
        bottom = 1590
        # Cropped image of above dimension 
        # (It will not change orginal image) 
        im1 = im.crop((left, top, right, bottom)) 
         # Shows the image in image viewer 
        im1.save("Makro Invoices/Makro Invoices/Converted/pp2.jpg")
        img  = cv2.imread('Makro Invoices/Makro Invoices/Converted/pp2.jpg', cv2.COLOR_RGB2GRAY)
        text=(pytesseract.image_to_string(img,config='outputbase digits'))
        text_list = text.split("\n")
        text_list = text_list
        print(text_list)
        for i in text_list:
            if len(i)>=4 and len(i)<=7:
                print(i)
                tot.append(i)
                dat["Total"] = max(tot)
            
            
        

<class 'list'>
N° Factura 721000050614
['N°', 'Factura', '721000050614']
Fecha Factura 02/08/2021
['Fecha', 'Factura', '02/08/2021']
| Total Factura| 574,88 €
['|', 'Total', 'Factura|', '574,88', '€']
['|', 'Total', 'Factura|', '574,88', '€']
Case Total
['|', 'Total', 'Factura|', '574,88', '€']
€
Factura emitida al amparo de la D.A. 4 del Real Decreto 1075/2017 de fecha 29 de Diciembre de 2017, por el que se modifica el Reglamento de
['Factura', 'emitida', 'al', 'amparo', 'de', 'la', 'D.A.', '4', 'del', 'Real', 'Decreto', '1075/2017', 'de', 'fecha', '29', 'de', 'Diciembre', 'de', '2017,', 'por', 'el', 'que', 'se', 'modifica', 'el', 'Reglamento', 'de']
['Factura', 'emitida', 'al', 'amparo', 'de', 'la', 'D.A.', '4', 'del', 'Real', 'Decreto', '1075/2017', 'de', 'fecha', '29', 'de', 'Diciembre', 'de', '2017,', 'por', 'el', 'que', 'se', 'modifica', 'el', 'Reglamento', 'de']
['Factura', 'emitida', 'al', 'amparo', 'de', 'la', 'D.A.', '4', 'del', 'Real', 'Decreto', '1075/2017', 'de', 'fecha', 

In [136]:
dat

{'Vat Number': 'B76546787',
 'Total': '574.88',
 'Date': ['04/08/2021'],
 'Invoice_Number': 'emitidaal'}

In [137]:
data  =  pd.DataFrame(dat,index=[0])
data['Vat Number']= data['Vat Number'].str.replace(' ', '')
print(dat)

{'Vat Number': 'B76546787', 'Total': '574.88', 'Date': ['04/08/2021'], 'Invoice_Number': 'emitidaal'}


In [278]:
bp = dat

In [279]:
data = pd.DataFrame(bp,index=[0])

In [280]:
data['Vat Number']= data['Vat Number'].str.replace('-', '')
data['Vat Number']= data['Vat Number'].str.replace(',', '')
data['Vat Number']= data['Vat Number'].str.replace('.', '')
data

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


,Vat Number,Total,Date,Invoice_Number
0,B76744861,"200,89",04/02/2020,


In [281]:
formats['last']= (formats['nif'].isin(data['Vat Number']))

In [282]:
df2 = formats.query("last == True")

In [283]:
df2

,formato,nombre,nif,last
9419,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
12062,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
12178,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
12780,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
16962,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
17485,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
18171,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
18852,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
18939,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
20388,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True


In [286]:
inv = 0
for i in df2['formato']:
    inv = i
pat = 'F1-1-01353'
factura = re.compile(inv)
mo = factura.search(text)
dat["Invoice_Number"]=mo.group()

In [287]:
dat

{'Vat Number': 'B76744861',
 'Total': '200,89',
 'Date': '04/02/2020',
 'Invoice_Number': 'F20/2473'}